# test-api.ipynb

Test API and helper functions

In [1]:
import os, sys
from typing import Optional, List, Dict
sys.path.append(os.path.join(".."))
import hjson
import docstring_parser
import inspect
from llms_wrapper.llms import LLMS
from llms_wrapper.config import update_llm_config

/home/johann/software/miniconda3/envs/llms_wrapper/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [17]:
import llms_wrapper
print(llms_wrapper.__version__)

0.1.11


In [2]:
config = dict(
    llms=[
        # OpenAI
        # https://platform.openai.com/docs/models
        dict(llm="openai/gpt-4o"),
        dict(llm="openai/gpt-4o-mini"),
        # dict(llm="openai/o1"),        # restricted
        # dict(llm="openai/o1-mini"),   # restricted
        # Google Gemini
        # https://ai.google.dev/gemini-api/docs/models/gemini
        dict(llm="gemini/gemini-2.0-flash-exp"),
        dict(llm="gemini/gemini-1.5-flash"),
        dict(llm="gemini/gemini-1.5-pro"),
        # Anthropic
        # https://docs.anthropic.com/en/docs/about-claude/models
        dict(llm="anthropic/claude-3-5-sonnet-20240620"),
        dict(llm="anthropic/claude-3-opus-20240229"),
        # Mistral
        # https://docs.mistral.ai/getting-started/models/models_overview/
        dict(llm="mistral/mistral-large-latest"),
        # XAI
        # dict(llm="xai/grok-2"),     # not mapped by litellm yet?
        dict(llm="xai/grok-beta"),
        # Groq
        # https://console.groq.com/docs/models
        dict(llm="groq/llama3-70b-8192"),
        dict(llm="groq/llama-3.3-70b-versatile"),
        # Deepseek
        # https://api-docs.deepseek.com/quick_start/pricing
        dict(llm="deepseek/deepseek-chat"),
    ],
    providers = dict(
        openai = dict(api_key_env="MY_OPENAI_API_KEY"),
        gemini = dict(api_key_env="MY_GEMINI_API_KEY"),
        anthropic = dict(api_key_env="MY_ANTHROPIC_API_KEY"),
        mistral = dict(api_key_env="MY_MISTRAL_API_KEY"),
        xai = dict(api_key_env="MY_XAI_API_KEY"),    
        groq = dict(api_key_env="MY_GROQ_API_KEY"),
        deepseek = dict(api_key_env="MY_DEEPSEEK_API_KEY"),
    )
)
_ = update_llm_config(config)

In [3]:
llms = LLMS(config)

In [4]:
llms.list_aliases()

['openai/gpt-4o',
 'openai/gpt-4o-mini',
 'gemini/gemini-2.0-flash-exp',
 'gemini/gemini-1.5-flash',
 'gemini/gemini-1.5-pro',
 'anthropic/claude-3-5-sonnet-20240620',
 'anthropic/claude-3-opus-20240229',
 'mistral/mistral-large-latest',
 'xai/grok-beta',
 'groq/llama3-70b-8192',
 'groq/llama-3.3-70b-versatile',
 'deepseek/deepseek-chat']

In [18]:
llms.cost_per_token("openai/gpt-4o")

(2.5e-06, 1e-05)

In [6]:
llms.max_input_tokens("openai/gpt-4o")

128000

In [7]:
llms.max_output_tokens("openai/gpt-4o")

16384

In [8]:
llms.make_messages("This is the query")

[{'content': 'This is the query', 'role': 'user'}]

## Cost, Tokens used

In [9]:
msg = llms.make_messages("What is a monoid?")

In [19]:
ret = llms.query("openai/gpt-4o", msg, return_cost=True)

In [20]:
ret

{'elapsed_time': 9.321770906448364,
 'cost': 0.0034425000000000002,
 'n_completion_tokens': 341,
 'n_prompt_tokens': 13,
 'n_total_tokens': 354,
 'answer': 'In abstract algebra, a monoid is an algebraic structure with a single associative binary operation and an identity element. More formally, a monoid is a set \\( M \\) equipped with a binary operation \\(\\ast: M \\times M \\to M\\) that satisfies the following properties:\n\n1. **Associativity**: For all elements \\( a, b, c \\) in \\( M \\), the equation \\((a \\ast b) \\ast c = a \\ast (b \\ast c)\\) holds. This means the order in which operations are performed does not change the result as long as the sequence of the operands is not changed.\n\n2. **Identity Element**: There exists an element \\( e \\) in \\( M \\) (called the identity element) such that for every element \\( a \\) in \\( M \\), the equations \\( e \\ast a = a \\ast e = a \\) hold. This means that there is a neutral element which, when combined with any other el

In [21]:
ret = llms.query("openai/gpt-4o", llms.make_messages("what is AI"), return_cost=True)
ret

{'elapsed_time': 3.084319829940796,
 'cost': 0.001445,
 'n_completion_tokens': 142,
 'n_prompt_tokens': 10,
 'n_total_tokens': 152,
 'answer': 'Artificial Intelligence, commonly referred to as AI, is a branch of computer science that involves creating systems capable of performing tasks that typically require human intelligence. These tasks include learning, reasoning, problem-solving, understanding natural language, perception, and even the ability to form plans or exhibit creativity. AI systems can range from simple algorithms used in everyday technology, such as virtual assistants and recommendation systems, to more complex neural networks used for advanced tasks like image and speech recognition, autonomous driving, and natural language processing. AI can be categorized into narrow AI, which is designed for specific tasks, and general AI, which aims to perform any intellectual task that a human can do, though the latter remains largely theoretical at this stage.',
 'error': '',
 'o

In [22]:
llms.cost("openai/gpt-4o")

0.0048875

In [25]:
llms.llms["openai/gpt-4o"].config

{'llm': 'openai/gpt-4o',
 'api_key_env': 'MY_OPENAI_API_KEY',
 'alias': 'openai/gpt-4o',
 '_cost': 0.0048875,
 '_elapsed_time': 12.40609073638916}

In [13]:
def func1(a: str, b: int, c: int = 1, d: Optional[List[Dict]] = None) -> str: 
    """
    This is the short description.

    Here we may have a longer description. This one can go over many lines

    :param str a: this is parameter a
    :param b: this is parameter b
    :type b: int
    :param c: some parameter c    
    :param d: some parameter d
    :return: what it returns
    :rtype: str
    """
    return "x"


In [14]:
llms.make_tooling(func1)

[{'type': 'function',
  'function': {'name': 'func1',
   'description': 'Here we may have a longer description. This one can go over many lines',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'str', 'description': 'this is parameter a'},
     'b': {'type': 'int', 'description': 'this is parameter b'},
     'c': {'type': 'int', 'description': 'some parameter c'},
     'd': {'type': 'Optional', 'description': 'some parameter d'}},
    'required': ['a', 'b']}}}]

In [ ]:
doc = docstring_parser.parse(func1.__doc__)

In [ ]:
doc.params[0]

In [ ]:
len(doc.params)